In [2]:
# Import required libraries
from sys import setprofile
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_geo.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                                     options=[
                                                              {'label': 'All sites', 'value': 'ALL'},
                                                              {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                              {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                              {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                              {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                             ],
                                                     value='ALL',
                                                     placeholder = 'Select a Launch Site here',
                                                     searchable = True, 
                                                     style={'width':'80%', 'padding':'3px', 'font-size':'20px', 'text-align-last':'center'}
                                                    ),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min = 0,
                                                max = 10000,
                                                step = 1000,
                                                marks = {
                                                    0: '0',
                                                    1000: '1000',
                                                    2000: '2000',
                                                    3000: '3000',
                                                    4000: '4000',
                                                    5000: '5000',
                                                    6000: '6000',
                                                    7000: '7000',
                                                    8000: '8000',
                                                    9000: '9000',
                                                    10000: '10000'
                                                },
                                                value = [min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(  Output(component_id='success-pie-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'))

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
def pie_chart(site_dropdown):
    fig = ''
    if site_dropdown == 'ALL':
        title_graph = "Success launches for all sites"
        fig = px.pie(spacex_df, values='class', names='Launch Site', title = title_graph)
    
    else:
        title_graph = f"Success launches for site {site_dropdown}"
        filtered_DD= spacex_df[spacex_df['Launch Site'] == site_dropdown]
        filtered_LS = filtered_DD.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_LS, values='class count', names='class', title=title_graph)
    return fig


@app.callback(  Output(component_id='success-payload-scatter-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'),
                Input(component_id="payload-slider", component_property="value"))

def payload_chart(site, payload):
    low,high = (payload[0], payload[1])
    df = spacex_df
    filtered_Spa = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]

    if site == 'ALL':
        title_sca = 'Payload vs. Outcome for ALL'
        fig = px.scatter(filtered_Spa,x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Payload vs. Outcome for All Sites')
        
    else:
        title_sca = ('Payload vs. Outcome for single launch site')
        filtered_sit= filtered_Spa[filtered_Spa['Launch Site'] == site]
        fig = px.scatter(filtered_sit,x="Payload Mass (kg)", y="class", color="Booster Version Category", title=title_sca)
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Sep/2021 21:48:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 21:48:27] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1622097690.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 21:48:27] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1622097690.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 21:48:27] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_3m1622097693.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 21:48:27] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1622097690.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 21:48:28] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1622097690.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 21:48:28] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_16_0m1622097691.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 21

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "E:\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "E:\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "E:\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "E:\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "E:\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "E:\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "E:\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "E:\anaconda3\lib\site-packages\d

127.0.0.1 - - [12/Sep/2021 21:48:33] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [12/Sep/2021 21:48:57] "POST /_dash-update-component HTTP/1.1" 200 -
